## 根据天凤的牌谱文件生成训练使用的dataset

In [1]:
import os, sys, re, json, torch
from components.Taikyoku_loader import Taikyoku_loader
from components.encoder import Encoder
from enum import Enum
import numpy as np

def default(obj):
    if isinstance(obj, Enum):
        return obj.value
    raise TypeError(f"{obj!r} is not JSON serializable")

os.makedirs("../../dataset/Pretaining", exist_ok=True)
dataset_path = "../../dataset/Tenhou/2011_nan_4_kuitan_aka"
# dataset_path = "../../dataset"
output_path = "../../dataset/Pretaining"

seq_buffer = []
masked_info_buffer = []
seq_batch_buffer = [] # 每一个元素是大小(64, max_seq_len, 55)的numpy数组
masked_info_batch_buffer = [] # 每一个元素是大小(64, max_seq_len, 1)的numpy数组
batch_index = 0
slice_num = 4

# 日志文件名的正则表达式，数字开头".log"结尾
file_name_pattern = re.compile(r"^\d+\.log$")
# file_name_pattern = re.compile(r"^test\.log$")
file_list = os.listdir(dataset_path)
file_list = [f for f in file_list if file_name_pattern.match(f)]

# 读取所有日志文件
for file in file_list:
    loader = Taikyoku_loader(os.path.join(dataset_path, file))

    for kyoku_index, kyoku in enumerate(loader.log):
        loader.reset(num=kyoku_index+1)
        # 从每局中随机提取slice_num个局面
        # random_index = np.random.choice(len(loader.log[kyoku_index]), slice_num, replace=False)
        random_index = np.random.choice(len(loader.log[kyoku_index]), min(slice_num, len(loader.log[kyoku_index])), replace=False)
        random_index.sort()
        # print(random_index)
        
        for i in range(max(random_index)+1):
            if i in random_index:
                output = loader.export_info()
                encoder = Encoder(output)
                encoded_seq, masked_info = encoder.encode()
                # print(encoded_seq.shape, masked_info.shape)
                seq_buffer.append(encoded_seq.T)
                masked_info_buffer.append(masked_info.T)
            if len(seq_buffer) == 64:
                # batch的形状为(64, max_seq_len, 55)
                max_seq_len = max([len(seq) for seq in seq_buffer])
                batch_seq = np.zeros((64, max_seq_len, 55))
                batch_masked_info = np.zeros((64, max_seq_len, 1))
                for i, seq in enumerate(seq_buffer):
                    batch_seq[i, :len(seq)] = seq
                for i, masked_info in enumerate(masked_info_buffer):
                    batch_masked_info[i, :len(masked_info)] = masked_info
                np.save(os.path.join(output_path, f"seq_{batch_index}.npy"), batch_seq)
                np.save(os.path.join(output_path, f"masked_info_{batch_index}.npy"), batch_masked_info)
                
                batch_index += 1
                if batch_index == 25000:
                    sys.exit()
                seq_buffer = []
                masked_info_buffer = []

            loader.step_forward()
        


KeyboardInterrupt: 

In [3]:
# 读取数据
mask = np.load(os.path.join(output_path, "masked_info_0.npy"))
batch = np.load(os.path.join(output_path, "seq_0.npy"))
print(mask.shape, batch.shape)

(64, 180, 1) (64, 180, 55)


In [4]:
os.path.pardir

'..'